In [1]:
import pandas as pd

In [ ]:
msp_data = pd.read_csv("../clean/mu_clean.csv",index_col=0)

msp_data['Fault Date'] = pd.to_datetime(msp_data['ZULU_Time'].str.split(" ", n = 2, expand = True)[1])

msp_data['Fault Time'] = pd.to_datetime(msp_data['ZULU_Time'].str.split(" ", n = 2, expand = True)[2], format = '%H:%M:%S:%f').dt.time

maf_data = pd.read_csv("../clean/maf_clean.csv", index_col = 0)

maf_data['Received Date'] = pd.to_datetime(maf_data['Received Date'])

maf_data['Completion Date'] = pd.to_datetime(maf_data['Completion Date'])

corrosion_actions = maf_data[maf_data['Corrosion']| maf_data['Bare Metal'] | maf_data['Corrosion Prevention Treatment']]

corrosion_actions["action_month"] = pd.DatetimeIndex(corrosion_actions['Completion Date']).month

corrosion_actions["action_year"] = pd.DatetimeIndex(corrosion_actions['Completion Date']).year

msp_data['action_year'] = pd.DatetimeIndex(msp_data['Fault Date']).year

combined_data = corrosion_actions.merge(msp_data[['Aircraft', 'action_year', 'MSP', 'Flight_Mode', 'Fault Date']], how = 'left', on = ['Aircraft', 'action_year'])

combined_data['before_window'] = combined_data['Completion Date'] - pd.DateOffset(months = 1)

combined_data['after_window'] = combined_data['Completion Date'] + pd.DateOffset(months = 1)

combined_data = combined_data[(combined_data['Fault Date'] > combined_data['before_window']) & (combined_data['Fault Date'] < combined_data['after_window'])]

combined_data['Pre'] = combined_data['Fault Date'] < combined_data['Completion Date']

combined_data = combined_data[(combined_data['Fault Date'] > combined_data['Completion Date']) | (combined_data['Fault Date'] < combined_data['Received Date'])]